# Chargement des packages et base de données
Ce code nécessite la version 2.8 de Tensorflow-text. 

In [ ]:
#chargement des packages
import tensorflow_hub as hub
import pandas as pd
import tensorflow_text as text
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import tensorflow as tf
import numpy as np

In [ ]:
#chargement du jeu de données
test_data=pd.read_parquet('mails_spam_clean.parquet')
test_data.head()

## Exploration de la base de données

In [ ]:
test_data.info() #2000 mails, aucune valeur manquante

In [ ]:
#renommer la base df
df= test_data
#il y autant de spams que de non-spams, la base est équilibrée
df['Spam'].value_counts()

In [ ]:
# creating 2 new dataframe as df_ham , df_spam

df_spam = df[df['Spam']==True]
df_ham = df[df['Spam']==False]

print("Taille des non-spams:", df_ham.shape)
print("Taille des spams:", df_spam.shape)

In [ ]:
# concatener les spams et les non spams, en vue de construire la base de test et la base d'apprentissage
df_balanced = pd.concat([df_spam , df_ham])

In [ ]:
# creer une variable "spam" qui vaut 1 pour "Spam", 0 pour "non-spam"
df_balanced['spam'] = df_balanced['Spam'].apply(lambda x:1 if x==True else 0)

In [ ]:
df_balanced.sample(4)

# Construction des bases d'apprentissage et de test
La variable cible y est la catégorie du mail : spam ou non. 
X contient les mots dans les mails correspondants à l'une ou l'autre catégorie. 
X_train et Y_train sont utilisées pour l'apprentissage et X_test et y_test pour le test. 

In [ ]:
# Split la base en X_train, X_test, y_train et y_test
from sklearn.model_selection import train_test_split
X_train, X_test , y_train, y_test = train_test_split(df_balanced['body'], df_balanced['spam'],
                                                    stratify = df_balanced['spam'])

# Construction du modèle de BERT



In [ ]:
# télécharger le préprocessing et l'encodage sur le site de BERT via Tensorflow
bert_preprocessor = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')
bert_encoder = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4')

In [ ]:
#on définit text_input pour y mettre les mots contenus dans le mail comme la couche 0
text_input = tf.keras.layers.Input(shape = (), dtype = tf.string, name = 'Inputs')
#on utilise la fonction bert_preprocessor() pour faire le processing des mots selon BERT
preprocessed_text = bert_preprocessor(text_input)
# embeed désigne le texte préprocessé encodé avec une fonction similaire de Bert pour l'encodage
embeed = bert_encoder(preprocessed_text)
# dropout donne l'entrée
dropout = tf.keras.layers.Dropout(0.1, name = 'Dropout')(embeed['pooled_output'])
# output donne la sortie de la couche 
outputs = tf.keras.layers.Dense(1, activation = 'sigmoid', name = 'Dense')(dropout)

In [ ]:
# creating final model
model = tf.keras.Model(inputs = [text_input], outputs = [outputs])

In [ ]:
#Voyons l'architecture du modèle
model.summary()

# Apprentissage

On construit et entraine notre modèle sur les bases X. 

In [ ]:
# creating final model
model = tf.keras.Model(inputs = [text_input], outputs = [outputs])

In [ ]:
# afficher les métriques accuracy, precison, et recall pour chaque époque
Metrics = [tf.keras.metrics.BinaryAccuracy(name = 'accuracy'),
           tf.keras.metrics.Precision(name = 'precision'),
           tf.keras.metrics.Recall(name = 'recall')]

In [ ]:
# compiler le modèle : on utilise l'optimizer adam et l'entropie binaire croisée pour la fonction de perte
model.compile(optimizer ='adam',
               loss = 'binary_crossentropy',
               metrics = Metrics)

In [ ]:
# phase d'apprentissage : 10 époques suffisent
history = model.fit(X_train, y_train, epochs = 10)

Le modèle s'améliore au fur à mesure des époques : l'accuracy, le recall et la precision augmentent tandis que la perte diminue. 
# Evaluation du modèle
Pour connaitre les métriques, on regarde les résultats pour la dernière couche. 

In [ ]:
model.evaluate(X_test,y_test)

In [ ]:
# y_pred est la prédiction de Bert entrainé sur nos données en fonction de X_test
y_pred = model.predict(X_test)
y_pred = y_pred.flatten()

In [ ]:
#si un mail a plus de 50% de probilité d'être un spam, alors on lui attribue 1, 0 sinon.
y_pred = np.where(y_pred > 0.5, 1, 0)

In [ ]:

#Afficher les métriques d'évaluation accuracy, precision, recall
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))
#Graphique : matrice de confusion
mat = confusion_matrix(y_test, y_pred)
sns.heatmap(mat, square=True, annot=True, fmt='d', cbar=True, cmap='coolwarm', linewidths=5)
#titres des axes 
plt.xlabel('Valeur prédite')
plt.ylabel('Valeur réelle')
#montrer le graphique
plt.show()

# Conclusion
Le modèle de BERT est très intéressant du point de vue de la vectorisation, mais son classifier pourrait être amélioré. 

L'accuracy est de 84 %. Le classifier prédit très bien les spams, mais a tendance à ranger des non-spams dans les spams. 